In [32]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/labs/amit/noamsh/repos/MM_2023')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [34]:
from clinical_predictions.clinical_data_loading import generate_refracrotines_dataset 
from clinical_predictions.utils import balanced_subsample

In [35]:
from clinical_predictions.evaluation import train_and_eval_model, generate_datasets_summerization

In [36]:
from functools import partial

import optuna
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split, cross_validate
from xgboost import XGBClassifier
import sklearn

def objective(trial, X_train, y_train):

    classifier_name = trial.suggest_categorical('classifier', ['RandomForest', 'XGBoost'])
    if classifier_name == 'SVC':
         svc_c = trial.suggest_float('svc_c', 1e-1, 1e3, log=True)
         model = sklearn.svm.SVC(C=svc_c, gamma='auto')
    elif classifier_name == 'Lasso':
         lasso_alpha = trial.suggest_float('lasso_alpha', 1e-1, 1e3, log=True)
         model = sklearn.linear_model.Lasso(alpha=lasso_alpha)
    elif classifier_name == 'RandomForest':
        rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32, log=True)
        rf_n_estimators = trial.suggest_int('rf_n_estimators', 2, 16, log=True)
        model = sklearn.ensemble.RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators)
    elif classifier_name == 'LogisticRegression':
        logistic_regression_c = trial.suggest_float('logistic_regression_c', 1e-1, 1e3, log=True)
        model = sklearn.linear_model.LogisticRegression(C=logistic_regression_c)
    else:
        param = {
            'booster': trial.suggest_categorical('xgb_booster', ['gbtree', 'dart']),
            'n_estimators': trial.suggest_int('xgb_n_estimators', 2, 16, log=True), 
            'max_depth': trial.suggest_int('xgb_max_depth', 1, 4), 
            # 'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True)
        }
        model = XGBClassifier(**param)
        
    # subsample = trial.suggest_categorical('subsample', [True, False])
    # if subsample:
    #     subsample_indexs = balanced_subsample(y_train)
    #     X_train = X_train.loc[subsample_indexs]
    #     y_train = y_train.loc[subsample_indexs]
    
    cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
    scores = cross_validate(model, X_train, y_train, cv=cv, scoring = ['accuracy', 'precision', 'f1', 'f1_weighted', 'f1_macro'])
    prec_alpha = 0.5
    mean_f1_prec_score = np.mean(scores['test_f1_weighted']  + scores['test_precision'] * prec_alpha)
    
    trial.set_user_attr(key="best_booster", value=model)
    return mean_f1_prec_score


def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_booster", value=trial.user_attrs["best_booster"])


def get_best_model_with_optuna(X_train, y_train, n_trials=30):
    study = optuna.create_study(direction='maximize')
    study.optimize(partial(objective, X_train=X_train, y_train=y_train), n_trials=n_trials, callbacks=[callback])
    best_model = study.user_attrs["best_booster"]
    best_trail = study.best_trial
    return best_model, best_trail

## loading

In [37]:
dataset_path = '/home/labs/amit/noamsh/repos/MM_2023/notebooks/tmp_data/zstat_MARS_SPID_20231130/MARS_SPID_exp_clin_nmf.xlsx'
raw_hospital_path = '/home/labs/amit/noamsh/repos/MM_2023/notebooks/tmp_data/zstat_MARS_SPID_20231130/Annonymized_CRF_BP_14112023S_SYW.xlsx'

In [38]:
raw_dataset = pd.read_excel(dataset_path)
raw_hospital_dataset = pd.read_excel(raw_hospital_path)
print(raw_dataset.shape)
# raw_dataset.head()

Workbook contains no default style, apply openpyxl's default


(262, 539)


In [39]:
print(raw_hospital_dataset.shape)
# raw_hospital_dataset.head()

(211, 111)


## general data filter

In [40]:
from clinical_predictions.clinical_data_loading import merge_transcriptom_data_to_raw_hospital

# remove no-transcriptom patients
# add post treatment columns from hospital
# add data to TAL_3 patients

dataset = merge_transcriptom_data_to_raw_hospital(raw_dataset, raw_hospital_dataset)

In [41]:
# path = '/home/labs/amit/noamsh/repos/MM_2023/notebooks/tmp_data/zstat_MARS_SPID_20231130/MARS_SPID_merged_with_hospital_231203.xlsx'
# dataset.to_excel(path)

### features selection

In [42]:
fish_cols = [col for col in raw_hospital_dataset.columns if "t(" in col or "del(" in col or col in ['1q21+', 'IGH rearrangement', 'Cytogenetics Risk (0=standard risk, 1=single hit, 2=2+ hits)']]
print(raw_hospital_dataset.shape)
dataset[fish_cols] = dataset[fish_cols].fillna(-1)
pd.concat([dataset[fish_col].value_counts().rename(fish_col) for fish_col in fish_cols], axis=1)


(211, 111)


,1q21+,del(1p),del(13q),del(17p),t(11:14),t(4:14),t(14:16),t(14:20),IGH rearrangement,"Cytogenetics Risk (0=standard risk, 1=single hit, 2=2+ hits)"
-1.0,54.0,55.0,57.0,54.0,53.0,54.0,56.0,59.0,59.0,59.0
1.0,41.0,13.0,17.0,21.0,25.0,7.0,3.0,1.0,20.0,26.0
0.0,40.0,67.0,61.0,60.0,57.0,74.0,76.0,75.0,56.0,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0


In [43]:
nmf_features = [f"X{i + 1}" for i in range(6)]
npc_composition = ["B", "B_Pro", "DC", "DC_IRF8", "Erythrocytes", "Fibro", "Mast", "Mf", "Mo", "Mo_CD16", "Neu_Pro",
                   "NK", "pDC", "T_Effector", "T_Effector_GZMB", "T_Naive", "UN"]
fish_cols = [col for col in raw_hospital_dataset.columns if "t(11:14)" == col or "del(" in col or col in ['1q21+', 'IGH rearrangement', 'Cytogenetics Risk (0=standard risk, 1=single hit, 2=2+ hits)']]

feats = nmf_features # + fish_cols
# feats = fish_cols

### label selection

In [44]:
datasets = {}
for treatment in ["Carfilzomib", "Lenalidomide", "Pomalidomide", "DARA", "CART", "Bortezomib", "Belantamab"]:

    # if treatment == "CART":
    #     non_ref_policy = "NDMM-POST_TREATMENT_REF"
    # else:
    #     non_ref_policy = "NDMM"

    non_ref_policy = "NON_EXPOSED"

    X,y = generate_refracrotines_dataset(dataset, treatment, non_ref_policy, feats)
    
    datasets[treatment] = (X, y)

In [45]:
for group in ["Triple.Ref" ,"Triple.Exp" ,"Penta.Ref" ,"Penta.Exp" ,"IMiD.Resistance.Len.Pom.Thali" ,"PI.Resistance.Bort.Carf.Ixa" ]:
    dataset_copy = dataset.copy()
    mask = ~ dataset_copy[group].isna()
    X = dataset_copy[mask][feats]
    y = dataset_copy[mask][group] == 1
    y = y.astype(int)

    datasets[group] = (X, y)

In [46]:
pd.concat([y.value_counts().rename(treatment) for treatment, (_, y) in datasets.items()], axis=1)

,Carfilzomib,Lenalidomide,Pomalidomide,DARA,CART,Bortezomib,Belantamab,Triple.Ref,Triple.Exp,Penta.Ref,Penta.Exp,IMiD.Resistance.Len.Pom.Thali,PI.Resistance.Bort.Carf.Ixa
0,115,71,103,93,134,55,127,82,71,105,95,63,55
1,20,60,31,46,3,51,7,28,39,5,15,47,55


In [47]:
good_treatments_data_sets = ["Bortezomib", "Lenalidomide", "Carfilzomib", "DARA", "Pomalidomide"] + ["Triple.Ref", "IMiD.Resistance.Len.Pom.Thali" ,"PI.Resistance.Bort.Carf.Ixa" ]

## model training

In [48]:
from sklearn.model_selection import train_test_split, GridSearchCV

good_splited_datasets = {}

for treatment in good_treatments_data_sets:
    X, y = datasets[treatment]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    good_splited_datasets[treatment] = X_train, X_test, y_train, y_test


In [49]:
pd.concat([y_test.value_counts().rename(treatment) for treatment , (_, _, _, y_test) in good_splited_datasets.items()], axis=1)

,Bortezomib,Lenalidomide,Carfilzomib,DARA,Pomalidomide,Triple.Ref,IMiD.Resistance.Len.Pom.Thali,PI.Resistance.Bort.Carf.Ixa
1,17,21,9,15,12,7,11,12
0,15,19,32,27,29,26,22,21


In [ ]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression, Lasso

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.svm import SVC

monitor = {}

for treatment in tqdm(good_splited_datasets):
    X_train, X_test, y_train, y_test = good_splited_datasets[treatment]
    model, best_trial = get_best_model_with_optuna(X_train, y_train, n_trials=25)
    monitor[treatment] = train_and_eval_model(X_train, X_test, y_train, y_test, model, extra_for_report=best_trial.params)


  0%|          | 0/8 [00:00<?, ?it/s][I 2023-12-05 14:26:31,114] A new study created in memory with name: no-name-3ad01f93-0cc2-4f43-8204-c50427ea9cd6
[I 2023-12-05 14:26:31,181] Trial 0 finished with value: 0.8286333965465144 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 6, 'rf_n_estimators': 8}. Best is trial 0 with value: 0.8286333965465144.
[I 2023-12-05 14:26:31,244] Trial 1 finished with value: 0.8595381221509116 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 9, 'rf_n_estimators': 8}. Best is trial 1 with value: 0.8595381221509116.
[I 2023-12-05 14:26:31,286] Trial 2 finished with value: 0.7755138224348751 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 14, 'rf_n_estimators': 3}. Best is trial 1 with value: 0.8595381221509116.
[I 2023-12-05 14:26:31,366] Trial 3 finished with value: 0.920029870083166 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 3, 'rf_n_estimators': 12}. Best is trial 3 with value: 0.92002987008316

In [ ]:
generate_datasets_summerization(monitor)

In [ ]:
list(generate_datasets_summerization(monitor)["extra"])

## train a stage model

In [ ]:
mask = ~ dataset["Stage"].isna()
X_stage = dataset[mask][feats]
y_stage = dataset[mask]["Stage"]
y_is_stage_3 = (y_stage == 3).astype(int)
y_0_is_1 = y_stage.apply(lambda x: x if x == 0 else x-1).astype(int)
y_stage.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stage, y_is_stage_3, test_size=0.2, random_state=42)

model, best_trial = get_best_model_with_optuna(X_train, y_train, n_trials=50)

monitor_stage = {}
monitor_stage["Stage"] = train_and_eval_model(X_train, X_test, y_train, y_test, model)
print(best_trial.params)

In [ ]:
generate_datasets_summerization(monitor_stage)

In [ ]:
monitor_stage["Stage"].model

In [ ]:
y_pred = monitor_stage["Stage"].y_pred
y_test = monitor_stage["Stage"].y_test
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
plt.show()

In [ ]:
### train no search with sub sample

In [ ]:
monitor_stage = {}

model = XGBClassifier(n_estimators=10, max_depth=3, learning_rate=1)

X_train, X_test, y_train, y_test = train_test_split(X_stage, y_is_stage_3, test_size=0.3, random_state=42)

monitor_stage["Stage"] = train_and_eval_model(X_train, X_test, y_train, y_test, model)

generate_datasets_summerization(monitor_stage)

In [ ]:
y_pred = monitor_stage["Stage"].y_pred
y_test = monitor_stage["Stage"].y_test
cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot()
plt.show()

In [ ]:
import shap

pred = model.predict(X_stage, output_margin=True)

explainer = shap.TreeExplainer(model)
explanation = explainer(X_stage)

shap_values = explanation.values
# make sure the SHAP values add up to marginal predictions
np.abs(shap_values.sum(axis=1) + explanation.base_values - pred).max()

In [ ]:
shap.plots.beeswarm(explanation)

In [ ]:
FP = y_test[(y_test==0).values & (y_pred==1).values] 
y_stage.loc[FP.index]

### FINAL EVALUATAION

In [ ]:
def bootstap_model_metric_distribution(model, X, y, n_samples, metric_name="f1-score"):
    baseline_f1_list = []
    model_f1_list = []
    for i in tqdm(range(n_samples)):
        X_train, X_test, y_train, y_test = train_test_split(X_stage, y_is_stage_3, test_size=0.3, random_state=42+i)
        exp_data = train_and_eval_model(X_train, X_test, y_train, y_test, model)
        model_f1_list.append(exp_data.report["1"][metric_name])
        baseline_f1_list.append(exp_data.report["baseline"]["1"][metric_name])
    return model_f1_list, baseline_f1_list

In [ ]:
def build_f1_df(model_f1_list, baseline_f1_list, contranst_name, metric_name):
    f1_df = pd.DataFrame({"model":["NMF model"] * len(model_f1_list) + ["baseline - random"] * len(baseline_f1_list),
                      metric_name :model_f1_list + baseline_f1_list})
    f1_df["contrast"] = contranst_name
    return f1_df

In [ ]:
f1_dfs = []
metric_name = "accuracy" # recall, precision, f1-score, accuracy
for treatment, exp_data in monitor.items():
    best_model = exp_data.model
    X = pd.concat([exp_data.X_train, exp_data.X_test], axis=0)
    y = pd.concat([exp_data.y_train, exp_data.y_test], axis=0)
    
    model_f1_list, baseline_f1_list = bootstap_model_metric_distribution(model, X, y, n_samples=20, metric_name=metric_name)
    
    f1_dfs.append(build_f1_df(model_f1_list, baseline_f1_list, treatment, metric_name))

model_f1_list, baseline_f1_list = bootstap_model_metric_distribution(XGBClassifier(n_estimators=10, max_depth=3, learning_rate=1),
                                                                 X_stage, y_is_stage_3, n_samples=20, metric_name=metric_name)
f1_dfs.append(build_f1_df(model_f1_list, baseline_f1_list, "Stage", metric_name))
f1_df = pd.concat(f1_dfs, axis=0)

In [ ]:
import plotly.express as px
fig = px.box(f1_df, color="model", x="contrast" ,y=metric_name,points="all",  width=800, height=400) #X for diffrent datasets
fig.update(layout_yaxis_range = [0,1])
fig.show()

### train STAGES WHITH 0 AND 1 Combibed

In [ ]:
# monitor_stage = {}

# model = XGBClassifier(n_estimators=10, max_depth=3, learning_rate=1)

# X_train, X_test, y_train, y_test = train_test_split(X_stage, y_0_is_1, test_size=0.3, random_state=42)
# monitor_stage["Stage"] = train_and_eval_model(X_train, X_test, y_train, y_test, model)

# monitor_stage["Stage"].report

In [ ]:

# y_pred = monitor_stage["Stage"].y_pred
# y_test = monitor_stage["Stage"].y_test
# cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
# disp.plot()
# plt.show()